In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from scipy.sparse.linalg import inv as sparse_inv
import matplotlib.animation as animation
from IPython.display import HTML

class Wave_Packet:
    def __init__(self, n_points, dt, sigma0=5.0, k0=1.0, x0=-150.0, x_begin=-200.0,
                 x_end=200.0, barrier_height=1.0, barrier_width=3.0, size=100):
        self.n_points = n_points
        self.sigma0 = sigma0
        self.k0 = k0
        self.x0 = x0
        self.dt = dt
        self.prob = np.zeros(n_points)
        self.barrier_width = barrier_width
        self.barrier_height = barrier_height

        # 1) Space discretization
        self.x, self.dx = np.linspace(x_begin, x_end, n_points, retstep=True)

        # 2) Initialization of the wave function to Gaussian wave packet
        norm = (2.0 * np.pi * sigma0**2)**(-0.25)
        self.psi = np.exp(-(self.x - x0)**2 / (4.0 * sigma0**2))
        self.psi = self.psi.astype(np.complex128)
        self.psi *= np.exp(1.0j * k0 * self.x)
        self.psi *= norm
        
        # 3) Setting up the potential barrier
        self.potential = np.array(
            [barrier_height if ((-100 < x < (-100 +barrier_width))|((-100+size)<x<(-100+size+barrier_width))) else 0.0 for x in self.x])
        
        self.potential += np.where(
            (self.x < x_begin + 15) | (self.x > x_end - 15),
            -1e22, 0.0)

        # 4) Creating the Hamiltonian
        h_diag = np.ones(n_points) / self.dx**2 + self.potential
        h_non_diag = np.ones(n_points - 1) * (-0.5 / self.dx**2)
        hamiltonian = sparse.diags([h_diag, h_non_diag, h_non_diag], [0, 1, -1])

        # 5) Computing the Crank-Nicolson time evolution matrix
        implicit = (sparse.eye(self.n_points) - dt / 2.0j * hamiltonian).tocsc()
        explicit = (sparse.eye(self.n_points) + dt / 2.0j * hamiltonian).tocsc()
        self.evolution_matrix = sparse_inv(implicit).dot(explicit).tocsr()

    def evolve(self):
        self.psi = self.evolution_matrix.dot(self.psi)
        self.prob = abs(self.psi)**2

        norm = sum(self.prob)
        self.prob /= norm
        self.psi /= norm**0.5

        return self.prob

    def compute_transmission(self):
        # Compute the transmission probability after evolution
        barrier_region = (self.x > 100 + self.barrier_width)  # Beyond the second barrier
        transmission = np.sum(self.prob[barrier_region])
        return transmission
    
    def has_crossed_barrier(self):
        # Define the region beyond the second barrier
        beyond_barrier = (self.x > 100 + self.barrier_width)
        # Check if a significant part of the probability is beyond the second barrier
        return np.sum(self.prob[beyond_barrier]) > 0.01  # Threshold for detection

def calculate_transmission_vs_energy(n_points, dt, sigma0, x0, x_begin, x_end, barrier_height, barrier_width, energy_range, size):
    transmissions = []
    energies = []

    for k0 in energy_range:
        # Create the wave packet with the current k0
        wave_packet = Wave_Packet(n_points=n_points, dt=dt, sigma0=sigma0, k0=k0, x0=x0,
                                  x_begin=x_begin, x_end=x_end, barrier_height=barrier_height,
                                  barrier_width=barrier_width, size=size)

        # Evolve the wave packet until it crosses the second barrier
        while not wave_packet.has_crossed_barrier():
            wave_packet.evolve()

        # Compute the transmission coefficient
        transmission = wave_packet.compute_transmission()
        energy = (k0**2) / 2  # Assuming natural units where m=1, ħ=1
        transmissions.append(transmission)
        energies.append(energy)

    return energies, transmissions

def plot_transmission_vs_energy(energies, transmissions):
    plt.figure(figsize=(10, 6))
    plt.plot(energies, transmissions, '-o', label='Transmission Probability')
    plt.xlabel('Incident Energy (a.u.)')
    plt.ylabel('Transmission Probability')
    plt.title('Transmission Probability vs. Incident Energy')
    plt.legend()
    plt.grid(True)
    plt.show()

# Parameters
n_points = 500
dt = 0.25
sigma0 = 5.0
x0 = -150.0
x_begin = -200.0
x_end = 200.0
barrier_height = 0.5
barrier_width = 5.0
size = 100
energy_range = np.linspace(0.1, 2.0, 20)  # Vary k0 to get different incident energies

energies, transmissions = calculate_transmission_vs_energy(
    n_points, dt, sigma0, x0, x_begin, x_end, barrier_height, barrier_width, energy_range, size
)
plot_transmission_vs_energy(energies, transmissions)
